# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_691262928f3081918bbcf988640b104b


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-WwLvdWg9ZVKYd3hAKPfr7Q


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-WwLvdWg9ZVKYd3hAKPfr7Q


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6029813181076801
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5984074448147717
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.5911156596667607
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5622353955032359
🤤 AI is making cinematic food commercials in minutes. The sizzling patty, the dripping cheese... no 
 Relevant score: 0.551242779477055


## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of 2023, generative AI continues to advance rapidly, with several key developments:

1. **Improved Language Models**: Newer language models have become more powerful, offering more nuanced and context-aware responses. These models are also being trained with a greater emphasis on ethical considerations and reducing biases.

2. **Multimodal Capabilities**: AI systems are increasingly combining text, image, and audio data. This allows them to perform complex tasks such as generating detailed image descriptions, creating images from textual prompts, and producing synchronized audio-visual content.

3. **Personalization**: Generative AI is being fine-tuned for individual users or specific applications, offering more personalized and context-relevant outputs.

4. **Creative Applications**: Artists and designers are using generative AI for creative projects, from music composition to digital art and video game design. AI is also being used to generate deepfake content more responsibly and creatively.

5. **AI in Research and Science**: Generative AI is playing a role in drug discovery, materials science, and other research fields by simulating complex processes and generating hypotheses for further investigation.

6. **Regulation and Ethics**: There is a growing focus on establishing ethical guidelines and regulatory frameworks to manage the impact of generative AI on society, particularly concerning misinformation and IP rights.

7. **Open Source Initiatives**: More open-source projects are emerging, allowing broader access to powerful generative models, fostering innovation, and community-based improvements.

These advancements are expanding the applications and implications of generative AI across various domains.

### File Search Response

In [13]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [14]:
display(Markdown(file_search_response.output_text))


It seems there are no specific details about the latest developments in generative AI in the uploaded files. However, I can provide a general overview based on current trends:

1. **Advanced Language Models**: Models like GPT-4 have improved in understanding and generating human-like text, with better context awareness and coherence.

2. **Multimodal Models**: These models can process and generate text, images, and other data types, allowing for more versatile applications.

3. **Ethical AI**: There is a growing focus on making AI more ethical and reducing biases in AI-generated content.

4. **AI in Creative Fields**: Generative AI is increasingly used in art, music, and content creation, enabling new forms of creativity.

5. **Improved Efficiency**: New techniques are being developed to make generative models more efficient, reducing the computational resources required.

If you have specific areas of interest, feel free to ask!

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [15]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [16]:
display(Markdown(web_search_response.output_text))

Here’s a comprehensive, in-depth overview (approx. 800 words) of the **latest developments in generative AI**, as of November 10, 2025. All major updates are accurately dated, clearly sourced, and professionally structured.

##  Developments in Generative AI Models & Tools

- **OpenAI’s GPT‑5 is now broadly deployed** in ChatGPT, Microsoft Copilot, and available via API. Launched on August 7, 2025, this multimodal model set a new benchmark in reasoning and multimodal functionality ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

- **Google DeepMind’s Veo 3.1**, the latest version of its text-to-video model (part of the Veo series), was released on **October 15, 2025**, offering improved video generation capabilities with synchronized audio ([en.wikipedia.org](https://en.wikipedia.org/wiki/Veo_%28text-to-video_model%29?utm_source=openai)).

- **Google’s “Nano Banana” (Gemini 2.5 Flash Image)** continues driving viral popularity. Released August 26, 2025, it enables highly photorealistic 3D-figurine-style image editing and is embedded in Gemini app and Vertex AI. It achieved over 10 million first-time users and 200 million image edits shortly after launch ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

##  Major Strategic Partnerships & Infrastructure Advances

- In a significant infrastructure expansion, **OpenAI and AWS signed a $38 billion strategic partnership on November 3, 2025**, granting OpenAI access to vast GPU resources and diversifying its compute infrastructure beyond Microsoft ([champaignmagazine.com](https://champaignmagazine.com/2025/11/09/ai-by-ai-weekly-top-5-november-3-9-2025/?utm_source=openai)).

- On **November 5, 2025**, **Apple sealed a deal to integrate Google’s Gemini models into Siri**, paying approximately $1 billion annually. This marks a major shift in Apple’s virtual assistant strategy ([champaignmagazine.com](https://champaignmagazine.com/2025/11/09/ai-by-ai-weekly-top-5-november-3-9-2025/?utm_source=openai)).

- On **November 6, 2025**, **Google unveiled “Ironwood,”** its seventh-generation TPU chip, offering 4× performance gains over predecessor. This reinforces Google’s hardware independence and strengthens AI computing infrastructure ([champaignmagazine.com](https://champaignmagazine.com/2025/11/09/ai-by-ai-weekly-top-5-november-3-9-2025/?utm_source=openai)).

##  Drug Discovery & Industry Applications

- On **November 7, 2025**, **Insilico Medicine**, a clinical-stage AI-driven drug discovery company, showcased a new portfolio of cardiometabolic drug assets at BIO Europe 2025, emphasizing the role of generative AI in advancing pharmaceuticals ([eurekalert.org](https://www.eurekalert.org/news-releases/1105110?utm_source=openai)).

- The **GENIUS project**, a large European collaboration, is advancing generative AI integration across the entire software development lifecycle, improving reliability and security in AI-assisted software engineering ([quantumzeitgeist.com](https://quantumzeitgeist.com/ai-genius-project-partners-advances-generative-integration-across-software/?utm_source=openai)).

- **The Entertainer**, a UAE-based savings platform, partnered with **Zero&One** to introduce generative AI features within its app and cloud ecosystem, improving customer engagement and personalization ([consultancy-me.com](https://www.consultancy-me.com/news/12033/savings-platform-the-entertainer-selects-zeroone-as-cloud-and-gen-ai-partner?utm_source=openai)).

##  Emerging AI Paradigms & Research Innovations

- **Google Research introduced “Nested Learning”** on November 7, 2025—a new continual learning paradigm framing models as nested optimization problems. It aims to address catastrophic forgetting by preserving performance on prior tasks while learning new ones ([research.google](https://research.google/blog/introducing-nested-learning-a-new-ml-paradigm-for-continual-learning/?utm_source=openai)).

- **The CHI 2025 “Tools for Thought” workshop (August 28, 2025)** explored how generative AI affects human cognition—covering areas like memory, critical thinking, and metacognition—and proposed frameworks for designing AI tools that augment and protect human thought ([arxiv.org](https://arxiv.org/abs/2508.21036?utm_source=openai)).

- **A recent information-theoretic framework** (published August 23, 2025) proposes introducing semantic-level metrics—like semantic entropy and channel capacity—to support more meaningful, human-aligned multimedia communication via generative AI, beyond traditional syntactic measures ([arxiv.org](https://arxiv.org/abs/2508.17163?utm_source=openai)).

- **CARING-AI**, a January 2025 research project, demonstrated context-aware AR instruction authored with generative AI avatars, enabling adaptive, location-aware learning through humanoid-avatar animations ([arxiv.org](https://arxiv.org/abs/2501.16557?utm_source=openai)).

##  Safety, Regulation & Ethical Dynamics

- **Tenable’s security researchers (today)** identified seven novel prompt injection methods that can exploit ChatGPT’s defaults to exfiltrate private data from chat histories—a serious emerging vulnerability ([csoonline.com](https://www.csoonline.com/article/4086965/researchers-trick-chatgpt-into-prompt-injecting-itself.html?utm_source=openai)).

- **OpenAI’s “Sora” video-gen AI app (2 days ago)** creates lifelike AI-generated videos quickly. Though powerful, it has sparked intense concern over safety and misinformation, particularly regarding its suitability for kids and potential for misuse ([abcnews.go.com](https://abcnews.go.com/GMA/Family/what-is-sora/story?id=127188940&utm_source=openai)).

##  Summary

The field of generative AI is advancing rapidly across multiple dimensions:

- **Model innovation**: GPT‑5, Gemini variants (including Nano Banana and Veo 3.1), and Alibaba’s Qwen series.
- **Compute & infrastructure**: Massive partnerships (OpenAI–AWS) and new hardware (Ironwood TPU).
- **Cross-sector applications**: From smarter drug discovery to integrated AI in finance, media, and cloud apps.
- **New learning paradigms**: Nested Learning and cognitive frameworks driving AI design.
- **Ethical and safety vigilance**: Prompt injection vulnerabilities and AI-generated media ethics are more critical than ever.

Let me know if you'd like a deeper dive into any specific area—e.g., GPT‑5’s architecture, Veo’s capabilities, ethical frameworks, or AI integration in healthcare or AR.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [17]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s a comprehensive, in-depth overview (approx. 800 words) of the **latest developments in genera

### Continue Query with Web Search

In [18]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(continue_search_response.output_text))

Here are several **different and recent developments** in the generative AI space, beyond what we've already covered:

• TIME launches the **TIME AI Agent** (Nov 10, 2025)  
TIME and Scale AI have introduced the "TIME AI Agent," a unified generative AI platform that enhances how readers interact with TIME's journalism. This system integrates language understanding, voice synthesis, translation, and search functions to offer dynamic engagement such as article summaries, audio reports, interactive exploration, and translations—all governed under TIME’s editorial standards for accuracy and transparency ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai)).

• OpenAI unveils **Aardvark**, a security-focused GPT-5–powered agent (late October 2025)  
OpenAI has rolled out “Aardvark,” an agentic cybersecurity tool currently in private beta. Leveraging GPT‑5, Aardvark autonomously scans code repositories for vulnerabilities—boasting about a 92% recall rate on seeded test cases—and has already contributed to ten officially tracked CVE-disclosed vulnerabilities ([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai)).

• Explosive launch success for **Sora on Android**  
OpenAI's video-generation app **Sora** premiered on Android on November 6, 2025, achieving roughly 470,000 downloads on its first day—surpassing the iOS debut by a significant margin (approx. 360,000 downloads), representing a 327% increase ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

• **Google Gemini AI** expands with “Deep Research” and **Nano Banana 2** on the horizon  
Gemini AI gains a new “Deep Research” capability, allowing it to leverage a user’s Gmail, Drive, and Chat content for context-aware and personalized responses ([medium.com](https://medium.com/%40CherryZhouTech/ai-news-november-1-7-2025-10-ai-advances-you-cant-miss-this-week-e2e86d331bd0?utm_source=openai)). Additionally, Google is reportedly preparing **Nano Banana 2**, an upgraded image generation model slated to launch soon, promising faster outputs, richer artistic styles, and built-in watermarking for transparency ([medium.com](https://medium.com/%40CherryZhouTech/ai-news-november-1-7-2025-10-ai-advances-you-cant-miss-this-week-e2e86d331bd0?utm_source=openai)).

• **Meta AI’s Vibes** expands to Europe  
Meta’s AI-generated short video platform **Vibes**—which uses generative AI to create every video seen and made on the platform—has now been launched in Europe via the Meta AI app, marking a major geographic expansion ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

• **DS STAR**, beyond traditional data science  
Google’s AI research team has introduced **DS STAR** (“Data Science Agent via Iterative Planning and Verification”), a multi-agent AI framework that can translate ambiguous business problems into executable Python code, handling diverse data formats like CSV, JSON, Markdown, and plain text—without human supervision ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

• **HeyGen releases AI Video Translator with Realistic Localization**  
HeyGen has debuted a next-generation AI video translator that enables foreign speakers to appear fluently speaking in other languages—complete with realistic expressions, lip movement, and tone—available via web, iOS, and API with free trial credits ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

• **Preview of Gemini 3 Pro** spotted on Vertex AI  
A “gemini-3-pro-preview-11-2025” model listing has appeared on Vertex AI, hinting at an imminent official launch. Expected to support a massive 1 million-token context window, it promises richer, longer-form interactions for developers and enterprises ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

• **Qualcomm announces AI200 and AI250 inference accelerators**  
Qualcomm has revealed two AI inference accelerators—AI200 (due 2026) and AI250 (due 2027)—powered by its Hexagon NPUs. Built for data centers, they support advanced features like micro-tile inferencing, 64-bit addressing, GenAI model encryption, and near-memory compute architecture, along with compatibility with AI frameworks like PyTorch and ONNX ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).

• **Adobe unveils AI-powered creative tools across its ecosystem**  
At Adobe MAX 2025, Adobe introduced a comprehensive suite of AI-powered enhancements across Express, Firefly, and Creative Cloud—including Firefly Image Model 5, end-to-end video generation, and integrations with tools like Photoshop, Premiere Pro, and even Google’s Nano Banana ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/adobe-launches-ai-audio-video-and-photo-tools-across-express-firefly-and-creative-cloud-brings-firefly-image-model-5-googles-nano-banana-and-more/articleshow/124876149.cms?utm_source=openai)).

---

### Summary of Key Trends

- Generative AI is increasingly integrated into core tools across journalism, cybersecurity, navigation, and creative design.
- Agentic AI—systems that autonomously perform tasks—is advancing fast, seen in tools like Aardvark and DS STAR.
- Advanced media synthesis continues to push boundaries, whether through improved image editors (Nano Banana 2), multimodal video apps (Sora), or translated AI-generated videos (HeyGen).
- Infrastructure providers, such as Qualcomm, are now building specialized hardware to meet generative AI's growing compute demands.

Let me know if you'd like a detailed breakdown of any of these developments—for instance, diving into how Aardvark benchmarks, Gemini 3 Pro’s likely capabilities, or Sora’s adoption trends.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [20]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [21]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, model releases, market trends, and emerging applications.

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model that integrates reasoning and non-reasoning capabilities under a unified interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and represents a significant leap in performance across benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **OpenAI o4‑mini**  
  Launched on **April 16, 2025**, this lightweight multimodal model supports both text and image inputs, including advanced features like whiteboard sketch analysis and chain-of-thought reasoning. A higher-accuracy variant, o4‑mini‑high, is available to paid-tier users. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai))

- **OpenAI GPT‑4.1**  
  Released on **April 14, 2025**, GPT‑4.1 (along with mini and nano variants) offers improved coding capabilities and is available to ChatGPT Plus and Pro subscribers. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai))

- **Google DeepMind’s Gemini 2.5 and “Nano Banana”**  
  Google’s Gemini 2.5 Pro and Flash models, featuring enhanced reasoning, coding, and “Deep Think” capabilities, became generally available on **June 17, 2025**. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai))  
  The image-focused variant, **Nano Banana** (Gemini 2.5 Flash Image), launched on **August 26, 2025**, and quickly went viral for its photorealistic 3D figurine-style image generation. It supports features like multi-image fusion, subject consistency, and SynthID watermarking. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Google Gemini Diffusion**  
  An experimental model that applies diffusion techniques to text generation, enabling simultaneous generation of text segments with mid-process error correction. It achieves speeds up to **1,479 tokens per second**, significantly faster than Gemini 2.5 Flash (~400 tps) and GPT‑4o (~150 tps). ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Alibaba Qwen3**  
  Released in **April 2025**, this open-source model family includes dense and Mixture-of-Experts variants (up to 235B parameters). It supports “thinking” and “non-thinking” modes and performs strongly on coding, math, and instruction-following benchmarks. ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Anthropic Claude Haiku 4.5**  
  A compact model launched in **October 2025**, delivering flagship-level reasoning and coding performance at 4–5× the speed and roughly one-third the cost of larger models. It features a 200K token context window and agentic capabilities like process identification. ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai))

- **OpenAI Sora (Video App)**  
  The Android version of Sora launched on **November 6, 2025**, achieving nearly **470,000 downloads on its first day**, outperforming its iOS debut by 327%. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  Emerging Applications & Ecosystem Integration

- **TIME AI Agent**  
  Launched **today**, this platform integrates language understanding, voice synthesis, translation, and search to deliver interactive, personalized journalism. Developed with Scale AI, it emphasizes editorial accuracy and transparency. ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **Google DS STAR**  
  Introduced in early November 2025, DS STAR is a multi-agent framework that converts ambiguous business problems into executable Python code, handling mixed-format data (CSV, JSON, Markdown, unstructured text) autonomously. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Meta Vibes Expansion**  
  Meta’s AI-generated short video platform, Vibes, expanded to Europe in early November 2025, offering TikTok-like experiences where every video is AI-generated. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Google Maps + Gemini Integration**  
  Google Maps is rolling out Gemini-powered voice assistance for intuitive location searches, with global availability expected in the coming weeks. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **HeyGen AI Video Translator**  
  Launched in early November 2025, this tool enables hyper-realistic localization of videos, matching tone, expressions, and lip movements. Available via web, iOS, and API with free trial credits. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **ClickUp 4.0**  
  Released in early November 2025, this update introduces AI agents and a redesigned UI, integrating task management, collaboration, messaging, scheduling, and enterprise search into a unified platform. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  Market Trends & Strategic Insights

- **Generative AI Market Growth**  
  A 2025 market report estimates triple-digit growth across GenAI hardware, foundation models, and development platforms in 2024, with over **US$400 billion** in expected AI-related spending in 2025. ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai))

- **Enterprise Adoption & Scaling**  
  The latest McKinsey Global Survey (published 5 days ago) shows that while many organizations are still piloting AI, **23%** are scaling agentic AI systems, and **39%** are experimenting with them. High performers are more likely to redesign workflows, define human validation processes, and allocate over **20% of digital budgets** to AI. ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai))

- **Responsible AI Practices**  
  UC Berkeley’s Responsible Use of Generative AI playbook (June 2025) outlines 10 actionable strategies—five for business leaders and five for product managers—to ensure ethical and responsible GenAI deployment. ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai))

- **Productivity Study**  
  A randomized controlled trial published in July 2025 found that experienced open-source developers using early-2025 AI tools took **19% longer** to complete tasks, suggesting that AI may introduce friction in certain workflows. ([metr.org](https://metr.org/blog/2025-07-10-early-2025-ai-experienced-os-dev-study/?utm_source=openai))

---

##  Summary: What’s Driving the Generative AI Landscape?

1. **Model Evolution**: The field is advancing rapidly with powerful multimodal models like GPT‑5, Gemini 2.5, and Claude Haiku 4.5, alongside experimental innovations like Gemini Diffusion and open-source Qwen3.

2. **Application Integration**: Generative AI is increasingly embedded into everyday tools—from journalism (TIME AI Agent) to navigation (Google Maps), video creation (Sora, HeyGen), and productivity platforms (ClickUp).

3. **Enterprise Momentum**: Organizations are moving beyond pilots, scaling agentic AI, and investing heavily in AI infrastructure and governance.

4. **Ethics & Productivity**: Responsible AI frameworks are gaining prominence, even as studies highlight potential productivity trade-offs in developer workflows.

---

If you'd like to explore any of these developments in more detail—such as technical capabilities of specific models, enterprise adoption strategies, or ethical frameworks—just let me know!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [12]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

with open('my_notes.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)


vs_691263b181a08191900c258ec6e4a346
file-VzsW7VcaVGSsejo4A1aMTa


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [22]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, academic findings, and industry releases to highlight key trends and breakthroughs.

---

##  Major Model Releases and Enhancements

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model accessible via ChatGPT, Microsoft Copilot, and the OpenAI API. It integrates reasoning and non-reasoning capabilities under a unified interface and achieved state-of-the-art performance across benchmarks at launch ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

- **OpenAI o4‑mini**  
  Launched on **April 16, 2025**, this lightweight reasoning model supports both text and image inputs, including whiteboard sketch analysis during chain-of-thought reasoning. A higher-accuracy variant, o4‑mini‑high, is available to paid users ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai)).

- **OpenAI GPT‑4.1**  
  Released on **April 14, 2025**, GPT‑4.1 (along with mini and nano variants) offers improved coding capabilities and is available via API and ChatGPT Plus/Pro ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai)).

- **Anthropic Claude 3.7 Sonnet**  
  Introduced in **February 2025**, this hybrid reasoning model features an “extended thinking mode” that enables self-reflection before responding, enhancing performance in math, physics, coding, and more. It’s available across all Claude plans, with the extended mode on paid tiers ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/anthropic-launches-advanced-ai-hybrid-reasoning-model-2025-02-24/?utm_source=openai)).

- **OpenAI gpt‑oss‑120b and gpt‑oss‑20b**  
  Released in **August 2025**, these are OpenAI’s first open-weight models since GPT‑2. The 20B model can run locally on consumer hardware (e.g., PCs with 16GB RAM or Snapdragon processors), while the 120B model requires more powerful GPUs ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/openai-launches-two-gpt-models-theyre-not-gpt-5-but-they-run-locally-on-snapdragon-pcs-and-nvidia-rtx-gpus?utm_source=openai)). AWS has since made these models available via Bedrock and SageMaker, offering significant cost-efficiency compared to other models ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/amazon-announces-first-ever-availability-of-openai-models-for-its-cloud-customers-company-says-the-addition-of-/articleshow/123125170.cms?utm_source=openai)).

---

##  Google DeepMind and Gemini Ecosystem

- **Gemini 2.5 Series**  
  Google’s Gemini 2.5 Pro and Flash models, released in early 2025, introduced advanced reasoning, coding capabilities, and a “Deep Think” mode. Gemini 2.5 Flash became the default model, with Flash‑Lite optimized for speed and cost-efficiency ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).

- **Nano Banana (Gemini 2.5 Flash Image)**  
  Launched on **August 26, 2025**, this image generation and editing model went viral for its photorealistic “3D figurine” outputs. It supports features like subject consistency, multi-image fusion, and SynthID watermarking, and quickly amassed over 10 million new users and 200 million image edits ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

---

##  Multimedia and Agentic AI Innovations

- **Unified Multimedia Models**  
  2025 saw a surge in models capable of handling text, image, audio, and video seamlessly:
  - **Google Veo 3** (June 2025): Generates video with synchronized sound effects and dialogue ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - **OpenAI Sora** (May 2025): Integrated into ChatGPT for conversational video generation ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - **Midjourney V7** (May 2025): Introduced 3D model generation and text-to-video tools ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).
  - **Runway Gen‑4** (April 2025): Added “Character Lock” to maintain character consistency across video scenes ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai)).

- **Agentic AI Tools**  
  - **Perplexity Comet**: Launched as the first agentic browser, capable of browsing tabs, voice commands, and task automation. Initially available to Perplexity Max users ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai)).
  - **xAI Grok 4**: Released mid-2025 with a 256K context window and state-of-the-art benchmark performance. Integrated into platforms like Cursor and LangChain ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai)).

---

##  Hardware and Infrastructure Advances

- **Qualcomm AI200 and AI250 Accelerators**  
  Announced in **October 2025**, these upcoming AI inference accelerators target data center workloads with features like micro-tile inferencing, 64-bit addressing, and Gen AI model encryption. The AI200 is slated for 2026 release, with AI250 following in 2027 ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).

---

##  Academic and Research Highlights

- **Diffusion Models in Protein Design**  
  A 2025 study highlights the success of diffusion-based generative models (e.g., RFDiffusion) in de novo protein design, outperforming traditional methods and reducing experimental costs ([arxiv.org](https://arxiv.org/abs/2504.16479?utm_source=openai)).

- **Semantic Information Theory for Multimedia AI**  
  A recent paper (August 2025) proposes a semantic-first information-theoretic framework for generative AI in multimedia communication, introducing concepts like semantic entropy and channel capacity ([arxiv.org](https://arxiv.org/abs/2508.17163?utm_source=openai)).

- **Quantum Generative Advantage**  
  A September 2025 study demonstrates generative quantum models on a 68-qubit superconducting processor, achieving tasks beyond classical simulation capabilities and marking a milestone in quantum-enhanced generative AI ([arxiv.org](https://arxiv.org/abs/2509.09033?utm_source=openai)).

---

##  Summary of Key Trends

- **Multimodal Integration**: Models now seamlessly handle text, image, audio, and video.
- **Reasoning and Reflection**: Features like “thinking modes” and self-reflection are becoming standard.
- **Accessibility and Efficiency**: Open-weight models and hardware optimizations are democratizing AI.
- **Agentic Capabilities**: AI is evolving from reactive tools to autonomous agents.
- **Cross-Domain Impact**: Generative AI is advancing in fields like protein design and quantum computing.

---

If you'd like to explore any of these developments in more detail—such as GPT‑5’s architecture, Nano Banana’s technical underpinnings, or the implications of quantum generative models—just let me know!